对embedding_lookup_sparse魔改，后归用于原生embedding_ops

In [1]:
import tensorflow as tf
from tensorflow.python.ops import math_ops,array_ops,sparse_ops
from tensorflow.python.framework import tensor_shape,sparse_tensor,dtypes
from tensorflow.python.framework import ops

### check weights and combiner
safe_embedding_lookup_sparse 处理后断言weights和ids的值维度、dense维度和下标均相同

In [2]:
sp_ids = tf.sparse.SparseTensor(indices=[[0,0],[0,1],[0,2],[1,0],[1,1],[2,0],[2,1],[3,0]],
                                    values=[123,234,11,1245,124,2121,124,2121],
                                    dense_shape=[5,3])
sp_ids,tf.sparse.to_dense(sp_ids)

(<tensorflow.python.framework.sparse_tensor.SparseTensor at 0x7f2f345d1390>,
 <tf.Tensor: shape=(5, 3), dtype=int32, numpy=
 array([[ 123,  234,   11],
        [1245,  124,    0],
        [2121,  124,    0],
        [2121,    0,    0],
        [   0,    0,    0]], dtype=int32)>)

In [3]:
sp_weights = tf.sparse.SparseTensor(indices=[[0,0],[0,1],[0,2],[1,0],[1,1],[2,0],[2,1],[3,0]],
                                    values=[1.0,1,1,1.0,0.5,2.0,0.5,2.0],
                                    dense_shape=[5,3])
sp_weights,tf.sparse.to_dense(sp_weights)

(<tensorflow.python.framework.sparse_tensor.SparseTensor at 0x7f2f345d1b10>,
 <tf.Tensor: shape=(5, 3), dtype=float32, numpy=
 array([[1. , 1. , 1. ],
        [1. , 0.5, 0. ],
        [2. , 0.5, 0. ],
        [2. , 0. , 0. ],
        [0. , 0. , 0. ]], dtype=float32)>)

In [4]:
sparse_ids, is_row_empty = sparse_ops.sparse_fill_empty_rows(sp_ids, 0)
sparse_weights, _ = sparse_ops.sparse_fill_empty_rows(sp_weights, 1.0)
tf.sparse.to_dense(sparse_ids,-99),tf.sparse.to_dense(sparse_weights,-99),is_row_empty

(<tf.Tensor: shape=(5, 3), dtype=int32, numpy=
 array([[ 123,  234,   11],
        [1245,  124,  -99],
        [2121,  124,  -99],
        [2121,  -99,  -99],
        [   0,  -99,  -99]], dtype=int32)>,
 <tf.Tensor: shape=(5, 3), dtype=float32, numpy=
 array([[  1. ,   1. ,   1. ],
        [  1. ,   0.5, -99. ],
        [  2. ,   0.5, -99. ],
        [  2. , -99. , -99. ],
        [  1. , -99. , -99. ]], dtype=float32)>,
 <tf.Tensor: shape=(5,), dtype=bool, numpy=array([False, False, False, False,  True])>)

### 对sp_ids，sp_weights进行处理

In [5]:
def sparse_slice(sparse_feature, max_length):
    if max_length is None:
        return sparse_feature
    max_length = tf.constant(max_length, dtype=tf.int64)
    return tf.sparse.slice(sparse_feature, [0, 0], [sparse_feature.dense_shape[0], max_length])

def SparseTensor_to_Dense(sp_input, max_length, default_id):
    process_feature = sparse_slice(sp_input, max_length)
    not_empty = tf.cast(tf.sparse.to_dense(process_feature, -1, name='check_empty') > -1, tf.float32)
    mask = tf.expand_dims(not_empty, axis=-1)
    sparse_hash_feature = tf.sparse.to_dense(process_feature, default_id, name='default_id')
    return sparse_hash_feature, mask

In [6]:
dense_ids, dense_mask = SparseTensor_to_Dense(sp_ids, max_length=None, default_id=0)
dense_ids,dense_mask

(<tf.Tensor: shape=(5, 3), dtype=int32, numpy=
 array([[ 123,  234,   11],
        [1245,  124,    0],
        [2121,  124,    0],
        [2121,    0,    0],
        [   0,    0,    0]], dtype=int32)>,
 <tf.Tensor: shape=(5, 3, 1), dtype=float32, numpy=
 array([[[1.],
         [1.],
         [1.]],
 
        [[1.],
         [1.],
         [0.]],
 
        [[1.],
         [1.],
         [0.]],
 
        [[1.],
         [0.],
         [0.]],
 
        [[0.],
         [0.],
         [0.]]], dtype=float32)>)

In [7]:
dense_weight, weight_mask = SparseTensor_to_Dense(sp_weights, max_length=None, default_id=1.0)
dense_weight,weight_mask

(<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
 array([[1. , 1. , 1. ],
        [1. , 0.5, 1. ],
        [2. , 0.5, 1. ],
        [2. , 1. , 1. ],
        [1. , 1. , 1. ]], dtype=float32)>,
 <tf.Tensor: shape=(5, 3, 1), dtype=float32, numpy=
 array([[[1.],
         [1.],
         [1.]],
 
        [[1.],
         [1.],
         [0.]],
 
        [[1.],
         [1.],
         [0.]],
 
        [[1.],
         [0.],
         [0.]],
 
        [[0.],
         [0.],
         [0.]]], dtype=float32)>)

In [8]:
ids, idx = array_ops.unique(tf.reshape(dense_ids, [-1, ]))
ids,idx

(<tf.Tensor: shape=(7,), dtype=int32, numpy=array([ 123,  234,   11, 1245,  124,    0, 2121], dtype=int32)>,
 <tf.Tensor: shape=(15,), dtype=int32, numpy=array([0, 1, 2, 3, 4, 5, 6, 4, 5, 6, 5, 5, 5, 5, 5], dtype=int32)>)

In [9]:
weights = tf.reshape(dense_weight, [-1, 1])
weights

<tf.Tensor: shape=(15, 1), dtype=float32, numpy=
array([[1. ],
       [1. ],
       [1. ],
       [1. ],
       [0.5],
       [1. ],
       [2. ],
       [0.5],
       [1. ],
       [2. ],
       [1. ],
       [1. ],
       [1. ],
       [1. ],
       [1. ]], dtype=float32)>

### 对去重后的ids获取embeddings
1. 执行embedding_lookup(params,ids)，获取embeddings
2. 此处假定获取为如下

In [10]:
emb = []
for index, values in enumerate(ids):
    emb.append(array_ops.ones([1,8])*(index))
embeddings = tf.concat(emb, axis=0)
embeddings

<tf.Tensor: shape=(7, 8), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1.],
       [2., 2., 2., 2., 2., 2., 2., 2.],
       [3., 3., 3., 3., 3., 3., 3., 3.],
       [4., 4., 4., 4., 4., 4., 4., 4.],
       [5., 5., 5., 5., 5., 5., 5., 5.],
       [6., 6., 6., 6., 6., 6., 6., 6.]], dtype=float32)>

In [11]:
embeddings = array_ops.gather(embeddings, idx)
embeddings,idx

(<tf.Tensor: shape=(15, 8), dtype=float32, numpy=
 array([[0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [2., 2., 2., 2., 2., 2., 2., 2.],
        [3., 3., 3., 3., 3., 3., 3., 3.],
        [4., 4., 4., 4., 4., 4., 4., 4.],
        [5., 5., 5., 5., 5., 5., 5., 5.],
        [6., 6., 6., 6., 6., 6., 6., 6.],
        [4., 4., 4., 4., 4., 4., 4., 4.],
        [5., 5., 5., 5., 5., 5., 5., 5.],
        [6., 6., 6., 6., 6., 6., 6., 6.],
        [5., 5., 5., 5., 5., 5., 5., 5.],
        [5., 5., 5., 5., 5., 5., 5., 5.],
        [5., 5., 5., 5., 5., 5., 5., 5.],
        [5., 5., 5., 5., 5., 5., 5., 5.],
        [5., 5., 5., 5., 5., 5., 5., 5.]], dtype=float32)>,
 <tf.Tensor: shape=(15,), dtype=int32, numpy=array([0, 1, 2, 3, 4, 5, 6, 4, 5, 6, 5, 5, 5, 5, 5], dtype=int32)>)

### 假设sp_weights非空

In [12]:
embeddings *= weights
embeddings

<tf.Tensor: shape=(15, 8), dtype=float32, numpy=
array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.],
       [ 3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.],
       [ 2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.],
       [ 5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.],
       [12., 12., 12., 12., 12., 12., 12., 12.],
       [ 2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.],
       [ 5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.],
       [12., 12., 12., 12., 12., 12., 12., 12.],
       [ 5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.],
       [ 5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.],
       [ 5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.],
       [ 5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.],
       [ 5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.]], dtype=float32)>

In [13]:
embeddings = tf.reshape(embeddings, [sp_ids.dense_shape[0],sp_ids.dense_shape[1],-1])
embeddings

<tf.Tensor: shape=(5, 3, 8), dtype=float32, numpy=
array([[[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.]],

       [[ 3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.],
        [ 2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.],
        [ 5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.]],

       [[12., 12., 12., 12., 12., 12., 12., 12.],
        [ 2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.],
        [ 5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.]],

       [[12., 12., 12., 12., 12., 12., 12., 12.],
        [ 5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.],
        [ 5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.]],

       [[ 5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.],
        [ 5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.],
        [ 5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.]]], dtype=float32)>

In [14]:
embeddings *= dense_mask
embeddings

<tf.Tensor: shape=(5, 3, 8), dtype=float32, numpy=
array([[[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.]],

       [[ 3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.],
        [ 2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]],

       [[12., 12., 12., 12., 12., 12., 12., 12.],
        [ 2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]],

       [[12., 12., 12., 12., 12., 12., 12., 12.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]], dtype=float32)>

In [15]:
tmp_shape = array_ops.stack([1, array_ops.shape(embeddings)[1],array_ops.shape(embeddings)[2]])
tmp_shape

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([1, 3, 8], dtype=int32)>

In [16]:
sparse_ids, is_row_empty = sparse_ops.sparse_fill_empty_rows(sp_ids, 0)

In [17]:
to_tile = array_ops.reshape(is_row_empty, [-1,1,1])
to_tile

<tf.Tensor: shape=(5, 1, 1), dtype=bool, numpy=
array([[[False]],

       [[False]],

       [[False]],

       [[False]],

       [[ True]]])>

In [18]:
is_row_empty = array_ops.tile(to_tile,tmp_shape)
is_row_empty

<tf.Tensor: shape=(5, 3, 8), dtype=bool, numpy=
array([[[False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False]],

       [[False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False]],

       [[False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False]],

       [[False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False]],

       [[ True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  Tru

In [19]:
result = array_ops.where(is_row_empty, array_ops.zeros_like(embeddings), embeddings)
result

<tf.Tensor: shape=(5, 3, 8), dtype=float32, numpy=
array([[[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.]],

       [[ 3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.],
        [ 2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]],

       [[12., 12., 12., 12., 12., 12., 12., 12.],
        [ 2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]],

       [[12., 12., 12., 12., 12., 12., 12., 12.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]], dtype=float32)>

In [20]:
original_shape = sp_ids.dense_shape
original_rank_dim = tensor_shape.dimension_value(sp_ids.dense_shape.get_shape()[0])
original_rank = (
    array_ops.size(original_shape)
    if original_rank_dim is None else original_rank_dim)
sp_ids = sparse_ops.sparse_reshape(sp_ids, [
    math_ops.reduce_prod(array_ops.slice(original_shape, [0], [original_rank - 1])),array_ops.gather(original_shape, original_rank - 1)])

In [21]:
# Reshape back from linear ids back into higher-dimensional dense result.
final_result = array_ops.reshape(result,array_ops.concat([
    array_ops.slice(math_ops.cast(original_shape, dtypes.int32), [0],[original_rank - 1]),array_ops.slice(array_ops.shape(result), [1], [-1])], 0))
final_result.set_shape(tensor_shape.unknown_shape(
    (tensor_shape.Dimension(original_rank_dim) - 1).value).concatenate(result.get_shape()[1:]))

In [22]:
final_result

<tf.Tensor: shape=(5, 3, 8), dtype=float32, numpy=
array([[[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.]],

       [[ 3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.],
        [ 2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]],

       [[12., 12., 12., 12., 12., 12., 12., 12.],
        [ 2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]],

       [[12., 12., 12., 12., 12., 12., 12., 12.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]], dtype=float32)>